In [1]:
import pandas as pd 
import re
from sqlalchemy import create_engine
##import pyodbc

#### Read customer data from stg_customer

#### Create a connection to SQL Srever 

In [2]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)
engine

Engine(mssql+pyodbc://@DB_SERVER/DB_NAME?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes)

#### Read Customer Data from stg_customer table

In [3]:
sql_stg_customer = '''SELECT *
                  FROM stg_customer'''

In [5]:
# store data to a dataframe 
stg_customer_df = pd.read_sql(sql_stg_customer,engine)
stg_customer_df.head()

,customer_name,contact_lastname,contact_firstname,phone,city,state,country,territory,deal_size
0,Land of Toys Inc.,Yu,Kwai,2125557818,NYC,NY,USA,None,Small
1,Reims Collectables,Henriot,Paul,26.47.1555,Reims,None,France,EMEA,Small
2,Lyon Souveniers,Da Cunha,Daniel,+33 1 46 62 7555,Paris,None,France,EMEA,Medium
3,Toys4GrownUps.com,Young,Julie,6265557265,Pasadena,CA,USA,None,Medium
4,Corporate Gift Ideas Co.,Brown,Julie,6505551386,San Francisco,CA,USA,None,Medium


In [6]:
len(stg_customer_df)

2823

#### Cleand Customer Data 

In [7]:
# check distinict country 
stg_customer_df['country'].value_counts()

country
USA            1004
Spain           342
France          314
Australia       185
UK              144
Italy           113
Finland          92
Norway           85
Singapore        79
Canada           70
Denmark          63
Germany          62
Sweden           57
Austria          55
Japan            52
Belgium          33
Switzerland      31
Philippines      26
Ireland          16
Name: count, dtype: int64

In [8]:
# check distinict customername values 
stg_customer_df['customer_name'].value_counts()

customer_name
Euro Shopping Channel           259
Mini Gifts Distributors Ltd.    180
Australian Collectors, Co.       55
La Rochelle Gifts                53
AV Stores, Co.                   51
                               ... 
Microscale Inc.                  10
Auto-Moto Classics Inc.           8
Royale Belge                      8
Atelier graphique                 7
Boards & Toys Co.                 3
Name: count, Length: 92, dtype: int64

In [9]:
# filter customername = 'Euro Shopping Channel' and check distinct
ESC = stg_customer_df[stg_customer_df ['customer_name'] == 'Euro Shopping Channel']
Grouped = ESC.groupby(['customer_name', 'contact_lastname', 'contact_firstname', 'country']).size().reset_index(name = 'COUNT')
Grouped

,customer_name,contact_lastname,contact_firstname,country,COUNT
0,Euro Shopping Channel,Freyre,Diego,Spain,259


In [11]:
# filter customername = 'Auto-Moto Classics Inc.'
AMCI = stg_customer_df[stg_customer_df ['customer_name'] == 'Auto-Moto Classics Inc.']
AMCI

,customer_name,contact_lastname,contact_firstname,phone,city,state,country,territory,deal_size
937,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Medium
1129,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Small
1217,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Medium
2088,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Medium
2554,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Small
2581,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Medium
2763,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Small
2790,Auto-Moto Classics Inc.,Taylor,Leslie,6175558428,Brickhaven,MA,USA,None,Small


##### Territory missing values

In [12]:
# Examine records having null TERRITORY value
Null_TERRITORY = stg_customer_df [stg_customer_df['territory'].isnull()]
Null_TERRITORY['country'].value_counts()

country
USA       1004
Canada      70
Name: count, dtype: int64

In [13]:
# Fill null 'territory' where country is USA or Canada with  North America
# Copy the dateframe 
stg_customer_df = stg_customer_df.copy()
stg_customer_df.loc[stg_customer_df['territory'].isna() & stg_customer_df['country'].isin(['USA', 'Canada']),'territory'] = 'North America'

In [14]:
stg_customer_df.head()

,customer_name,contact_lastname,contact_firstname,phone,city,state,country,territory,deal_size
0,Land of Toys Inc.,Yu,Kwai,2125557818,NYC,NY,USA,North America,Small
1,Reims Collectables,Henriot,Paul,26.47.1555,Reims,None,France,EMEA,Small
2,Lyon Souveniers,Da Cunha,Daniel,+33 1 46 62 7555,Paris,None,France,EMEA,Medium
3,Toys4GrownUps.com,Young,Julie,6265557265,Pasadena,CA,USA,North America,Medium
4,Corporate Gift Ideas Co.,Brown,Julie,6505551386,San Francisco,CA,USA,North America,Medium


##### State missing values

In [15]:
# examine null STATE values
null_STATE =  stg_customer_df[stg_customer_df['state'].isnull()]
null_STATE['country'].value_counts()

country
Spain          342
France         314
UK             118
Italy          113
Finland         92
Norway          85
Singapore       79
Denmark         63
Germany         62
Sweden          57
Austria         55
Belgium         33
Switzerland     31
Philippines     26
Ireland         16
Name: count, dtype: int64

In [16]:
# Replace NaN State values with N/A
stg_customer_df['state']  = stg_customer_df['state'].fillna('N/A')

In [17]:
# Check update
stg_customer_df[stg_customer_df['country'].isin(['Spain','France','UK','Italy', 'Ieland'])]

,customer_name,contact_lastname,contact_firstname,phone,city,state,country,territory,deal_size
1,Reims Collectables,Henriot,Paul,26.47.1555,Reims,N/A,France,EMEA,Small
2,Lyon Souveniers,Da Cunha,Daniel,+33 1 46 62 7555,Paris,N/A,France,EMEA,Medium
6,Daedalus Designs Imports,Rance,Martine,20.16.1555,Lille,N/A,France,EMEA,Small
9,Auto Canal Petit,Perrier,Dominique,(1) 47.55.6555,Paris,N/A,France,EMEA,Medium
14,La Rochelle Gifts,Labrune,Janine,40.67.8555,Nantes,N/A,France,EMEA,Medium
...,...,...,...,...,...,...,...,...,...
2814,"AV Stores, Co.",Ashworth,Victoria,(171) 555-1555,Manchester,N/A,UK,EMEA,Small
2815,La Rochelle Gifts,Labrune,Janine,40.67.8555,Nantes,N/A,France,EMEA,Small
2818,Euro Shopping Channel,Freyre,Diego,(91) 555 94 44,Madrid,N/A,Spain,EMEA,Small
2820,Euro Shopping Channel,Freyre,Diego,(91) 555 94 44,Madrid,N/A,Spain,EMEA,Medium


##### Drop duplicates

In [18]:
# Drop duplicates
stg_customer_df= stg_customer_df.drop_duplicates()

In [19]:
len(stg_customer_df)

254

##### Format phone number

In [20]:
stg_customer_df['country'].value_counts()

country
USA            97
France         35
Australia      15
Spain          14
UK             12
Norway          9
Finland         9
Germany         8
Italy           8
Canada          7
Sweden          6
Denmark         6
Singapore       6
Austria         5
Japan           5
Belgium         4
Philippines     3
Ireland         3
Switzerland     2
Name: count, dtype: int64

In [21]:
country_dial_codes = {
    'USA': '+1',
    'France': '+33',
    'Australia': '+61',
    'Spain': '+34',
    'UK': '+44',
    'Norway': '+47',
    'Finland': '+358',
    'Germany': '+49',
    'Italy': '+39',
    'Canada': '+1',
    'Sweden': '+46',
    'Denmark': '+45',
    'Singapore': '+65',
    'Austria': '+43',
    'Japan': '+81',
    'Belgium': '+32',
    'Philippines': '+63',
    'Ireland': '+353',
    'Switzerland': '+41'
}

In [22]:
df_phone_country = stg_customer_df[['country','phone']].head(30)
#type(phone_country)
df_phone_country

,country,phone
0,USA,2125557818
1,France,26.47.1555
2,France,+33 1 46 62 7555
3,USA,6265557265
4,USA,6505551386
5,USA,6505556809
6,France,20.16.1555
7,Norway,+47 2267 3215
8,USA,6505555787
9,France,(1) 47.55.6555


In [23]:
stg_customer_df['phone'].isna().sum()

np.int64(0)

In [24]:
def extract_local_number(df):
    # Step 1: Map country to country code
    df['country code'] = df['country'].map(country_dial_codes)

    # Step 2: Clean each phone number
    def clean(row):
        phone = row['phone']
        country_code = row['country code'].replace('+', '')  # e.g., '33'

        # Remove anything in parentheses
        phone = re.sub(r'\(.*?\)', '', phone)

        # Remove formatting characters: spaces, dashes, dots
        phone = re.sub(r'[.\-\s]', '', phone)

        # Remove leading '+'
        phone = phone.lstrip('+')

        # If phone starts with the country code, remove it
        if phone.startswith(country_code):
            phone = phone[len(country_code):]

        # Remove leading 0 (optional, based on local dialing conventions)
        phone = phone.lstrip('0')

        return phone

    # Apply the clean function
    df['phone_local'] = df.apply(clean, axis=1)
    return df


In [25]:
stg_customer_df = stg_customer_df.copy()
stg_customer_df = stg_customer_df.reset_index(drop = True)
extract_local_number (stg_customer_df)

,customer_name,contact_lastname,contact_firstname,phone,city,state,country,territory,deal_size,country code,phone_local
0,Land of Toys Inc.,Yu,Kwai,2125557818,NYC,NY,USA,North America,Small,+1,2125557818
1,Reims Collectables,Henriot,Paul,26.47.1555,Reims,N/A,France,EMEA,Small,+33,26471555
2,Lyon Souveniers,Da Cunha,Daniel,+33 1 46 62 7555,Paris,N/A,France,EMEA,Medium,+33,146627555
3,Toys4GrownUps.com,Young,Julie,6265557265,Pasadena,CA,USA,North America,Medium,+1,6265557265
4,Corporate Gift Ideas Co.,Brown,Julie,6505551386,San Francisco,CA,USA,North America,Medium,+1,6505551386
...,...,...,...,...,...,...,...,...,...,...,...
249,Alpha Cognac,Roulet,Annette,61.77.6555,Toulouse,N/A,France,EMEA,Small,+33,61776555
250,Boards & Toys Co.,Young,Leslie,3105552373,Glendale,CA,USA,North America,Small,+1,3105552373
251,La Rochelle Gifts,Labrune,Janine,40.67.8555,Nantes,N/A,France,EMEA,Large,+33,40678555
252,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Stavern,N/A,Norway,EMEA,Large,+47,7989555


###### Drop phone number

In [26]:
stg_customer_df.drop('phone', axis=1, inplace = True)

In [27]:
# rename phone_local column to phone
stg_customer_df = stg_customer_df.rename(columns = {'phone_local': 'phone'})
stg_customer_df

,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,country code,phone
0,Land of Toys Inc.,Yu,Kwai,NYC,NY,USA,North America,Small,+1,2125557818
1,Reims Collectables,Henriot,Paul,Reims,N/A,France,EMEA,Small,+33,26471555
2,Lyon Souveniers,Da Cunha,Daniel,Paris,N/A,France,EMEA,Medium,+33,146627555
3,Toys4GrownUps.com,Young,Julie,Pasadena,CA,USA,North America,Medium,+1,6265557265
4,Corporate Gift Ideas Co.,Brown,Julie,San Francisco,CA,USA,North America,Medium,+1,6505551386
...,...,...,...,...,...,...,...,...,...,...
249,Alpha Cognac,Roulet,Annette,Toulouse,N/A,France,EMEA,Small,+33,61776555
250,Boards & Toys Co.,Young,Leslie,Glendale,CA,USA,North America,Small,+1,3105552373
251,La Rochelle Gifts,Labrune,Janine,Nantes,N/A,France,EMEA,Large,+33,40678555
252,Baane Mini Imports,Bergulfsen,Jonas,Stavern,N/A,Norway,EMEA,Large,+47,7989555


In [28]:
stg_customer_df.rename(columns = {'country code': 'country_code'}, inplace = True)

In [29]:
stg_customer_df.head(4)

,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,country_code,phone
0,Land of Toys Inc.,Yu,Kwai,NYC,NY,USA,North America,Small,+1,2125557818
1,Reims Collectables,Henriot,Paul,Reims,N/A,France,EMEA,Small,+33,26471555
2,Lyon Souveniers,Da Cunha,Daniel,Paris,N/A,France,EMEA,Medium,+33,146627555
3,Toys4GrownUps.com,Young,Julie,Pasadena,CA,USA,North America,Medium,+1,6265557265


#### Load Data to DimCustomer Using Incremental Load Approach

##### 1) Load Customer Data from DimCustomer Table

In [31]:
# load DimCustomer table into a DataFrame
sql_DimCustomer = '''SELECT *
                  FROM  DimCustomer'''

In [32]:
dim_customer_df = pd.read_sql(sql_DimCustomer, engine)
dim_customer_df.head(7)

,customer_id,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,phone,country_code


##### 2) Perform stg_customer left join DimCustomer tables to find new records 

In [41]:
# Merge on a composite key columns 
merged_customer_df = stg_customer_df.merge(dim_customer_df, on = ['customer_name','contact_lastname',
                                                         'contact_firstname', 'country', 'deal_size']
                                 , how = 'left', indicator = True, suffixes = ('','_dim'))
merged_customer_df.head()

,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,country_code,phone,customer_id,city_dim,state_dim,territory_dim,phone_dim,country_code_dim,_merge
0,Land of Toys Inc.,Yu,Kwai,NYC,NY,USA,North America,Small,+1,2125557818,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Reims Collectables,Henriot,Paul,Reims,N/A,France,EMEA,Small,+33,26471555,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,Lyon Souveniers,Da Cunha,Daniel,Paris,N/A,France,EMEA,Medium,+33,146627555,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Toys4GrownUps.com,Young,Julie,Pasadena,CA,USA,North America,Medium,+1,6265557265,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Corporate Gift Ideas Co.,Brown,Julie,San Francisco,CA,USA,North America,Medium,+1,6505551386,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [42]:
# Filter records that are only in staging but not in DimCustomer 
new_customer_records_df = merged_customer_df[merged_customer_df['_merge']== 'left_only']
new_customer_records_df.head()

,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,country_code,phone,customer_id,city_dim,state_dim,territory_dim,phone_dim,country_code_dim,_merge
0,Land of Toys Inc.,Yu,Kwai,NYC,NY,USA,North America,Small,+1,2125557818,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Reims Collectables,Henriot,Paul,Reims,N/A,France,EMEA,Small,+33,26471555,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,Lyon Souveniers,Da Cunha,Daniel,Paris,N/A,France,EMEA,Medium,+33,146627555,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Toys4GrownUps.com,Young,Julie,Pasadena,CA,USA,North America,Medium,+1,6265557265,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Corporate Gift Ideas Co.,Brown,Julie,San Francisco,CA,USA,North America,Medium,+1,6505551386,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [43]:
# drop columns from DimCustomer and keep only staging data
new_customer_records_df= new_customer_records_df[stg_customer_df.columns]
new_customer_records_df.head()

,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,country_code,phone
0,Land of Toys Inc.,Yu,Kwai,NYC,NY,USA,North America,Small,+1,2125557818
1,Reims Collectables,Henriot,Paul,Reims,N/A,France,EMEA,Small,+33,26471555
2,Lyon Souveniers,Da Cunha,Daniel,Paris,N/A,France,EMEA,Medium,+33,146627555
3,Toys4GrownUps.com,Young,Julie,Pasadena,CA,USA,North America,Medium,+1,6265557265
4,Corporate Gift Ideas Co.,Brown,Julie,San Francisco,CA,USA,North America,Medium,+1,6505551386


##### 3) Insert new records into DimCustomer  MS Server SQL Table 

In [46]:
# re-order columns 
re_ordered_cols = ['customer_name', 'contact_lastname', 'contact_firstname', 'city',
       'state', 'country', 'territory', 'deal_size','phone', 'country_code']

In [48]:
new_customer_records_df= new_customer_records_df[re_ordered_cols]
new_customer_records_df.head()

,customer_name,contact_lastname,contact_firstname,city,state,country,territory,deal_size,phone,country_code
0,Land of Toys Inc.,Yu,Kwai,NYC,NY,USA,North America,Small,2125557818,+1
1,Reims Collectables,Henriot,Paul,Reims,N/A,France,EMEA,Small,26471555,+33
2,Lyon Souveniers,Da Cunha,Daniel,Paris,N/A,France,EMEA,Medium,146627555,+33
3,Toys4GrownUps.com,Young,Julie,Pasadena,CA,USA,North America,Medium,6265557265,+1
4,Corporate Gift Ideas Co.,Brown,Julie,San Francisco,CA,USA,North America,Medium,6505551386,+1


In [50]:
# insert new records to the target table
total_inserted_customer_records = new_customer_records_df.to_sql('DimCustomer', con = engine, if_exists = 'append', index = False)
print(total_inserted_customer_records)

45
